In [ ]:
import pandas as pd
import fastparquet
import pyarrow.parquet as pq

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)
pd.set_option('mode.chained_assignment', None)

In [ ]:
import os

# Altera o diretório atual para a pasta "/home/user/my_dir"
os.chdir("C:/Users/leandro.menezes/Downloads")

In [ ]:
pel = pd.read_parquet('pnl_consolidado (5).parquet')
pel.shape

In [ ]:
for colum in pel.columns:
    print(colum)

In [ ]:
janeiro = pd.read_excel('planilha',sheet_name = 'sheet 1')

In [ ]:
for colum in janeiro.columns:
    print(colum)

In [ ]:
pel = pd.concat([pel,janeiro])

In [ ]:
pel.shape

In [ ]:
pel.head()

In [ ]:
for coluna in pel.columns:
    print(coluna)


In [ ]:
df_pivot = pel[pel['oringin_state'].isin(['SP', 'MG', 'SC']) & pel['destination_state'].isin(['SP', 'MG', 'SC'])].pivot_table(
    values=['carrier_cost', 'tracking_code'], index=['oringin_state', 'destination_state'], aggfunc={'carrier_cost': sum, 'tracking_code': 'size'})

In [ ]:
df_pivot

In [ ]:
df_pivot2 = pel[
    (pel['oringin_state'] == 'SP') & (pel['destination_state'] == 'SP')
    | (pel['oringin_state'] == 'MG') & (pel['destination_state'] == 'MG')
    | (pel['oringin_state'] == 'SC') & (pel['destination_state'] == 'SC')
].pivot_table(
    values=['carrier_cost', 'tracking_code'],
    index=['oringin_state', 'destination_state'],
    aggfunc={'carrier_cost': sum, 'tracking_code': 'size'})

In [ ]:
df_pivot2

In [ ]:
df_pivot2.to_excel('Consulta Robson volume por destino.xlsx')

## Adicionando month aos indices

In [ ]:
df_pivot2 = pel[
    (pel['oringin_state'] == 'SP') & (pel['destination_state'] == 'SP')
    | (pel['oringin_state'] == 'MG') & (pel['destination_state'] == 'MG')
    | (pel['oringin_state'] == 'SC') & (pel['destination_state'] == 'SC')
].pivot_table(
    values=['carrier_cost', 'tracking_code'],
    index=['oringin_state', 'destination_state', 'month'], 
    aggfunc={'carrier_cost': sum, 'tracking_code': 'size'})

In [ ]:
df_pivot2

In [ ]:
df_pivot2.to_excel('envios_uf_to_uf.xlsx')

In [ ]:
pel['month'] = pd.to_datetime(pel['purchase_timestamp']).dt.month

In [ ]:
pel.head()

In [ ]:
pnl_15_01['margem'] = (pnl_15_01['nd_log'] - pnl_15_01['carrier_cost'])

In [ ]:
pnl_15_01['margem']

In [ ]:
pnl_15_01['nd_log']

In [ ]:
pnl_15_01['Nome da Origem'].nunique()

## loggi, por volume de tracking quais as regiões eu estou pagando mais 

In [ ]:
nomes_origens = pnl_15_01['Nome da Origem'].unique()

for nome in nomes_origens:
    print(nome)

In [ ]:
loggi_encontrados = pnl_15_01[pnl_15_01['Nome da Origem'] == ' - Loggi']


In [ ]:
loggi_encontrados.shape

In [ ]:
loggi_encontrados.head()

In [ ]:
df = loggi_encontrados[['tracking_code', 'oringin_state', 'carrier_cost']]

In [ ]:
df_pivot = df.pivot_table(values='carrier_cost', index='oringin_state', aggfunc="sum")


In [ ]:
df_pivot

In [ ]:
pip install --upgrade pandas


In [ ]:
df_pivot = df.pivot_table(values=['carrier_cost', 'tracking_code'], index='oringin_state', aggfunc={'carrier_cost': sum, 'tracking_code': 'size'})

In [ ]:
df_pivot

In [ ]:
df_pivot['carrier_cost_per_tracking'] = df_pivot['carrier_cost'] / df_pivot['tracking_code']

In [ ]:
df_pivot

In [ ]:
df_pivot['carrier_cost_per_tracking'] = df_pivot['carrier_cost'] / df_pivot['tracking_code']


In [ ]:
df_pivot = df_pivot.sort_values('carrier_cost', ascending=False)

In [ ]:
df_pivot = df_pivot.rename(columns={'tracking_code': 'Quantidade de Trackings', 'carrier_cost_per_tracking': 'CMU'})


In [ ]:
df_pivot

In [ ]:
df2 = loggi_encontrados[['tracking_code', 'oringin_state', 'carrier_cost','cubed_weight_range']]

In [ ]:
df_pivot222 = df2.pivot_table(values=['tracking_code','carrier_cost'], index='cubed_weight_range', aggfunc={'tracking_code': 'size', 'carrier_cost':sum})

In [ ]:
nomes_origens = pnl_15_01['cubed_weight_range'].unique()

for nome in nomes_origens:
    print(nome)

## Movendo coluna de lugar

In [ ]:
df_pivot222 = df_pivot222.reindex(columns=['tracking_code', 'carrier_cost'])

In [ ]:
df_pivot222

In [ ]:
for i in range(len(df_pivot222)):
    if df2.loc[i, 'cubed_weight_range'] == 'greater than 29000':
        df2.loc[i, 'cubed_weight_range'] = 10
    else:
        df2.loc[i, 'cubed_weight_range'] = int(df2.loc[i, 'cubed_weight_range'].split('-')[0]) + 2

In [ ]:
df_pivot222

In [ ]:
df_pivot222.index = df_pivot222.index.map(lambda x: 10 if x == "greater than 29000" else int(x.split('-')[0]) + 1)

In [ ]:
df_pivot222


In [ ]:
df_pivot222.index = df_pivot222.index.map(lambda x: 10 if x == "greater than 29000" else int(x.split('-')[0]) + 2)


In [ ]:
df_pivot222.to_excel('tabela_analise.xlsx')

In [ ]:
tabela = df_pivot222

In [ ]:
tabela.index = tabela.index.map(lambda x: pd.to_numeric(x.split('-')[0]) + 1 if isinstance(x, str) else x)

In [ ]:
tabela = pd.read_excel('tabela_analise.xlsx')
tabela = pd.DataFrame(tabela)
tabela